#### Transforms examples

* The two obvious perspectives to this data are "resource" and "customer / user" perspective. 
    * It makes sense to model these separately as they are different use cases and slightly different requirements
        * Both are demonstrated below
    * User id has a high cardinality, so we do daily aggregation 


In [42]:

import duckdb
duckdb.sql(
"""
WITH currency_exchange AS (
    SELECT 
        'EUR' AS currency, 
        1.00 AS from_eur, 
        1.00 AS to_eur
    UNION ALL 
    SELECT 'USD', 1.108984, 0.901726
    UNION ALL
    SELECT 'JPY', 164.658011, 0.006073
),
aggs AS (
    SELECT
        DATE_TRUNC('hour', timestamp) AS dt,
        region,
        service_tier,
        operation_type,
        resource_type,

        /* The credits do seem to be credits and not monetary cost
            However, as a nice exercise lets do a "conversion" */
        SUM(-credit_usage * to_eur) AS spend,
        
        SUM(-credit_usage) AS credit_usage,

        SUM(success) AS successful_operations,
        COUNT(*) AS operations,

        /* note that these two cannot be summed so not true facts */
        COUNT(DISTINCT user_id) AS distinct_users,
        COUNT(DISTINCT resource_id) AS distinct_resources

    FROM '../../../../../data/billing.csv'
    LEFT JOIN currency_exchange
    USING(currency)
    GROUP BY
        dt,
        region,
        service_tier,
        operation_type,
        resource_type
)
SELECT * FROM aggs
"""
)

┌─────────────────────┬─────────┬──────────────┬────────────────┬───────────────┬────────────────────┬────────────────────┬───────────────────────┬────────────┬────────────────┬────────────────────┐
│         dt          │ region  │ service_tier │ operation_type │ resource_type │       spend        │    credit_usage    │ successful_operations │ operations │ distinct_users │ distinct_resources │
│      timestamp      │ varchar │   varchar    │    varchar     │    varchar    │       double       │       double       │        int128         │   int64    │     int64      │       int64        │
├─────────────────────┼─────────┼──────────────┼────────────────┼───────────────┼────────────────────┼────────────────────┼───────────────────────┼────────────┼────────────────┼────────────────────┤
│ 2025-05-01 02:00:00 │ us-chi1 │ enterprise   │ write          │ container     │      5372.17043826 │  8999.880000000001 │                   174 │        188 │            186 │                188 │
│ 202

Second we have the user perspective.

I have included user_dims as a sample of creating some user specific dimensions from the existing data, I don't know whether any of these could even be relevant

In [ ]:
duckdb.sql(
"""
WITH user_dims AS (
    SELECT 
        MIN(timestamp) AS first_seen,
        MAX(timestamp) AS last_seen,

        /* Generally these I guess could be created bi-side but leaving them here for convenience */
        CURRENT_LOCALTIMESTAMP() - MIN(timestamp) AS user_age,
        CURRENT_LOCALTIMESTAMP() - MAX(timestamp) AS since_last_visit,
        user_id
    FROM '../../../../../data/billing.csv'
    GROUP BY user_id
),
aggs AS (
    SELECT
        CAST(timestamp AS DATE) AS date,
        user_id,
        first_seen,
        last_seen,
        user_age,
        since_last_visit,
        
        /* Some of these could be pre-pivoted for convenience
          depending on use case. However, seems pretty sparse */
        service_tier,
        operation_type,
        resource_type,

        SUM(-credit_usage) AS credit_usage,
        SUM(success) AS successful_operations,
        COUNT(*) AS operations,

        /* note that cannot be summed so not true facts */
        COUNT(DISTINCT resource_id) AS distinct_resources

    FROM '../../../../../data/billing.csv'
    LEFT JOIN user_dims
    USING(user_id)
    GROUP BY
        date,
        user_id,
        first_seen,
        last_seen,
        user_age,
        since_last_visit,
        region,
        service_tier,
        operation_type,
        resource_type,
)
SELECT * FROM aggs
""")

┌────────────┬─────────┬─────────────────────┬─────────────────────┬──────────────────────┬──────────────────────┬──────────────┬────────────────┬───────────────┬───────────────────┬───────────────────────┬────────────┬────────────────────┐
│    date    │ user_id │     first_seen      │      last_seen      │       user_age       │   since_last_visit   │ service_tier │ operation_type │ resource_type │   credit_usage    │ successful_operations │ operations │ distinct_resources │
│    date    │  int64  │      timestamp      │      timestamp      │       interval       │       interval       │   varchar    │    varchar     │    varchar    │      double       │        int128         │   int64    │       int64        │
├────────────┼─────────┼─────────────────────┼─────────────────────┼──────────────────────┼──────────────────────┼──────────────┼────────────────┼───────────────┼───────────────────┼───────────────────────┼────────────┼────────────────────┤
│ 2025-05-01 │   19840 │ 2025-05-01 

I was also thinking of modeling e.g. usage tier as a slowly changing dimension of user, but this does not seem to be a holistic user "plan" but rather resource dependent.
Leaving it here as an exercise still.

In [33]:
duckdb.sql("""   
    WITH ranked_tiers AS (
  SELECT
    user_id,
    service_tier,
    timestamp,
    ROW_NUMBER() OVER (PARTITION BY user_id, service_tier ORDER BY timestamp) AS rn,
    LAG(timestamp) OVER (PARTITION BY user_id ORDER BY timestamp) AS prev_ts,
    LEAD(timestamp) OVER (PARTITION BY user_id ORDER BY timestamp) AS next_ts
  FROM '../../../../../data/billing.csv'
),
deduped_changes AS (
  SELECT
    user_id,
    service_tier,
    timestamp AS valid_from,
    next_ts AS valid_to,
    CASE WHEN next_ts IS NULL THEN true ELSE false END AS is_current
  FROM (
    SELECT *,
           LAG(service_tier) OVER (PARTITION BY user_id ORDER BY timestamp) AS prev_tier
    FROM ranked_tiers
  ) t
  WHERE prev_tier IS DISTINCT FROM service_tier  -- only include real changes
     OR prev_tier IS NULL                         -- include the first state
)
SELECT * FROM deduped_changes ORDER BY user_id, valid_from
""")

┌─────────┬──────────────┬─────────────────────┬─────────────────────┬────────────┐
│ user_id │ service_tier │     valid_from      │      valid_to       │ is_current │
│  int64  │   varchar    │      timestamp      │      timestamp      │  boolean   │
├─────────┼──────────────┼─────────────────────┼─────────────────────┼────────────┤
│   10000 │ enterprise   │ 2025-05-01 00:19:11 │ 2025-05-01 00:23:49 │ false      │
│   10000 │ basic        │ 2025-05-01 00:32:06 │ 2025-05-01 00:50:26 │ false      │
│   10000 │ free         │ 2025-05-01 00:50:26 │ 2025-05-01 01:06:05 │ false      │
│   10000 │ pro          │ 2025-05-01 01:06:05 │ 2025-05-01 01:09:33 │ false      │
│   10000 │ basic        │ 2025-05-01 01:09:33 │ 2025-05-01 01:13:28 │ false      │
│   10000 │ enterprise   │ 2025-05-01 01:13:28 │ 2025-05-01 01:13:40 │ false      │
│   10000 │ free         │ 2025-05-01 01:27:40 │ 2025-05-01 01:28:30 │ false      │
│   10000 │ enterprise   │ 2025-05-01 01:28:30 │ 2025-05-01 01:36:14 │ false